# Classifying YouTube Videos for Humpback Whale Encounters - Keras CNN-RNN

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import pickle
import glob
import cv2
import os

from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
import seaborn as sns

import wandb

In [4]:
#ngc workspace path (where we keep our data)
workspace_path = '/mount/data'

In [ ]:
#start wandb session for metric logging
wandb.login() 

wandb.init(project="whale-classification-inception")

# Inception V3 (CNN-RNN) 

## Hyperparameters

In [15]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_NUM_FRAMES = 461
NUM_FEATURES = 2048

In [11]:
#read all frames in for 1 video from workspace 'frames' directory
frames = []
for i in range(461):
    if i % 100 == 0:
        print(i)
    
    #read in .jpg file as array for video clip 0000
    img = cv2.imread(workspace_path + f'/frames/clip_0000_frame_{i}.jpg')
    frames.append(img)

frames = np.array(frames)
frames.shape

(461, 224, 224, 3)

461 frames of size 224 x 224 with RGB color channels

# Load Frames + Extract Features

In [33]:
from feature_extraction import load_frames, prepare_all_videos
from cnn import CNN

In [28]:
frames = load_frames('video_0033.mp4', MAX_NUM_FRAMES)
frames.shape

(1, 461, 224, 224, 3)

In [36]:
ConvNet = CNN(IMG_SIZE)
feature_extractor = ConvNet.InceptionV3()
feature_extractor

2022-07-08 13:18:08.244923: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-08 13:18:13.471221: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14649 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:06:00.0, compute capability: 7.0
2022-07-08 13:18:13.475227: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14649 MB memory:  -> device: 1, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:0b:00.0, compute capability: 7.0


87924736/87910968 [==============================] - 4s 0us/step


In [38]:
data = pd.read_csv(workspace_path + '/downloaded_videos.csv')
y = data.pop('relevant')
X = data

In [ ]:
y

In [42]:
X_trial = X[0:5]
y_trial = y[0:5]
(frame_features, frame_masks), labels = prepare_all_videos(X_trial, y_trial, MAX_NUM_FRAMES, NUM_FEATURES, feature_extractor)

#the error is probably bc of the label formatting

2022-07-08 13:23:02.272486: I tensorflow/stream_executor/cuda/cuda_dnn.cc:379] Loaded cuDNN version 8400


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_10212/3293004085.py", line 3, in <cell line: 3>
    (frame_features, frame_masks), labels = prepare_all_videos(X_trial, y_trial, MAX_NUM_FRAMES, NUM_FEATURES, feature_extractor)
  File "/workspace/youtube-humpback-whale-classifier/classification/feature_extraction.py", line 59, in prepare_all_videos
    labels = y['relevant'].astype(int)
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/series.py", line 942, in __getitem__
    return self._get_value(key)
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/series.py", line 1051, in _get_value
    loc = self.index.get_loc(label)
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/range.py", line 388, in get_loc
    raise KeyError(key)
KeyError: 'relevant'

During handling of the above exception